# ReadMe

<p><strong>This notebook</strong> contains the codes for evaluating the performance of the model in terms of different probabilistic and deterministic metrics.</p>

<p><strong>The deterministic metrics are:</strong></p>
<ul>
    <li><em>Mean Bias Error (MBE)</em></li>
    <li><em>coefficient of determination (R2)</em></li>
</ul>

<p><strong>The probabilistic metrics are:</strong></p>
<ul>
    <li><em>Continuous Ranked Probability Score (CRPS)</em></li>
    <li><em>Continuous Ranked Probability Skill Score (CRPSS)</em></li>
</ul>

<p><strong>Python version:</strong> 3.11.3</p>

<p><strong>Utilized packages:</strong></p>
<ul>
    <li><code>numpy (1.24.3)</code></li>
    <li><code>pandas (2.0.1)</code></li>
    <li><code>matplotlib, seaborn (3.7.1, 0.12.2)</code></li>
    <li><code>ensverif (0.1.0)</code></li>
</ul>

<p><strong>Guide to the notebook:</strong></p>

<p>Each section is separated by a markdown cell with a title. The sections are:</p>

<h3>Imports</h3>
<p><strong>Import the required packages and define the helper functions.</strong></p>

<h3>Variables</h3>
<p><strong>Define the variables used in the notebook.</strong></p>

<h3>Read and process data</h3>
<p><strong>Read the data and process it into the required format.</strong></p>

<h3>Deterministic metrics</h3>
<p><strong>Calculate the deterministic metrics.</strong></p>

<h3>Probabilistic metrics</h3>
<p><strong>Calculate the probabilistic metrics.</strong></p>

<h3>Plot the results</h3>
<p><strong>Plot the results into bar charts.</strong></p>

<p><strong>The notebook is designed to be run from top to bottom.</strong></p>

# Imports

In [ ]:
# <<< imports >>> -----------------------------------------------------------

from pathlib import Path
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

import ensverif as ev

# make fonts bold in plots
plt.rcParams["font.weight"] = "bold"
# ___________________________________________________________________ <<< >>>



## functions

In [ ]:
# <<< functions >>> ---------------------------------------------------------

def calculate_crps(df, num_scenarios=200, ens_columns_prefix='scn_', obs_column='Value'):
    """
    Calculate the Continuous Ranked Probability Score (CRPS) for a given DataFrame.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing ensemble forecasts and observations.
    num_scenarios : int
        Number of ensemble scenarios to consider.
    ens_columns_prefix : str, optional
        Prefix for column names representing ensemble forecasts. Default is 'scn_'.
    obs_column : str, optional
        Column name representing observations. Default is 'Value'.

    Returns
    -------
    pd.DataFrame
        DataFrame with the CRPS result.

    """

    # Extract the ensemble forecasts
    ens = df[[ens_columns_prefix + str(i) for i in range(num_scenarios)]]

    # Extract the observations
    obs = df[obs_column]

    # Calculate CRPS
    crps_result = ev.crps_hersbach_decomposition(ens, obs)[0]
    
    # Return the CRPS result as a DataFrame
    return pd.DataFrame({'CRPS': [crps_result]})


def benchmark_crps(df, ens_num=200, ens_factor=0.25, obs_column='Value'):
    """
    Calculate the benchmark CRPS for a given DataFrame.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing observations.
    ens_num : int
        Number of ensemble forecasts to generate.
    ens_factor : float
        Scaling factor for ensemble forecasts.
    obs_column : str, optional
        Column name representing observations. Default is 'Value'.

    Returns
    -------
    pd.DataFrame
        DataFrame with the benchmark CRPS result.

    """

    ens_shape = (df.shape[0], ens_num)

    # Define the ensemble based on mean and standard deviation
    ens_range = np.linspace(
        np.nanmean(df[obs_column]) - np.nanstd(df[obs_column]) * ens_factor,
        np.nanmean(df[obs_column]) + np.nanstd(df[obs_column]) * ens_factor,
        ens_num,
        endpoint=True
    )

    # Generate the ensemble
    ens = np.tile(ens_range, ens_shape[0]).reshape(ens_shape)


    # Extract the observations
    obs = df[obs_column]

    # Calculate CRPS
    crps_result = ev.crps_hersbach_decomposition(ens, obs)[0]

    # Return the CRPS result as a DataFrame
    return pd.DataFrame({'Benchmark CRPS': [crps_result]})


def get_units(variable, metric):
    """
    Get the units for a given variable and metric.

    Parameters
    ----------
    variable : str
        Variable name.
    metric : str
        Metric name.

    Returns
    -------
    str or None
        Units corresponding to the variable and metric combination, or None if not found.

    """

    # Define the units using a dictionary
    units_mapping = {
        ("Soil Moisture", "CRPS"): "%",
        ("Soil Moisture", "CRPSS"): "-",
        ("Soil Moisture", "r2"): "-",
        ("Soil Moisture", "MBE"): "%",
        ("Soil Temperature", "CRPS"): "°C",
        ("Soil Temperature", "CRPSS"): "-",
        ("Soil Temperature", "r2"): "-",
        ("Soil Temperature", "MBE"): "°C",
        ("Percolation", "CRPS"): "mm",
        ("Percolation", "CRPSS"): "-",
        ("Percolation", "r2"): "-",
        ("Percolation", "MBE"): "mm",
        ("Snow Depth", "CRPS"): "cm",
        ("Snow Depth", "CRPSS"): "-",
        ("Snow Depth", "r2"): "-",
        ("Snow Depth", "MBE"): "cm",
    }

    # Lookup the units in the dictionary
    units = units_mapping.get((variable, metric))

    return units

def get_axis_limits(variable, metric):

    # define the units using a dictionary
    axis_limits_mapping = {
        ("Soil Moisture", "CRPS"): (0, 10),
        ("Soil Moisture", "CRPSS"): (-0.9, 0.9),
        ("Soil Moisture", "r2"): (0, 1),
        ("Soil Moisture", "MBE"): (-8, 8),
        ("Soil Temperature", "CRPS"): (0, 4),
        ("Soil Temperature", "CRPSS"): (-0.9, 0.9),
        ("Soil Temperature", "r2"): (0, 1),
        ("Soil Temperature", "MBE"): (-4, 4),
        ("Percolation", "CRPS"): (0, 1.0),
        ("Percolation", "CRPSS"): (-0.9, 0.9),
        ("Percolation", "r2"): (0, 1),
        ("Percolation", "MBE"): (-0.7, 0.7),
        ("Snow Depth", "CRPS"): (0, 5),
        ("Snow Depth", "CRPSS"): (-0.9, 0.9),
        ("Snow Depth", "r2"): (0, 1),
        ("Snow Depth", "MBE"): (-4, 4),
    }

    # lookup the units in the dictionary
    axis_limits = axis_limits_mapping.get((variable, metric))

    return axis_limits
# ___________________________________________________________________ <<< >>>

# Variables

In [ ]:
# <<< variable definition >>> -----------------------------------------------

paths_dict = {

    # path to the output of the model runs
    # (This is produced by the notebook: ...)
    "svs_output": Path(
        "./output/DailyAveraged_SVS_outputs.csv"
    ),

    # path to percolation observations
    "obs_percolation": Path(
        "./data/percolation_daily_20180701_20210630.csv"
    ),

    # path to soil observations
    "obs_soil": Path(
        "./data/state_vars_daily_20180701_20210630.csv"
    ),

    # path to daily meteo observations
    "obs_meteo": Path(
        "./data/DailyAveraged_MeteoObs.csv"
    ),
}

# dates for the evaluation period
START_DATE = pd.to_datetime("2019-07-01")
END_DATE = pd.to_datetime("2021-06-30")

# label for the date column in all of the dataframes
DATE_LABEL = "date"

# SVS layers corresponding to the sensor at 7.5 cm depth
svs_layers = [3, 4]

# number of ensemble scenarios
N_ENS = 200

# create winter periods for evaluation of snow depth simulations
# 2019-11-01 to 2020-04-01 (inclusive) + 2020-11-01 to 2021-04-01 (inclusive)
first_winter = pd.date_range("2019-11-01", "2020-04-01", freq="D")
second_winter = pd.date_range("2020-11-01", "2021-04-01", freq="D")
# ___________________________________________________________________ <<< >>>

# Read and process the data

In [ ]:
# <<< Read Data >>> ------------------------------------------------------------

# read the output of the model runs
svs_output = pd.read_csv(paths_dict["svs_output"], parse_dates=[DATE_LABEL])

# read the observed percolation data
obs_percolation = pd.read_csv(paths_dict["obs_percolation"], parse_dates=[DATE_LABEL])

# read the observed soil state variables
obs_soil = pd.read_csv(paths_dict["obs_soil"], parse_dates=[DATE_LABEL])

# read the observed meteorological data (we only need snow depth)
obs_meteo = pd.read_csv(paths_dict["obs_meteo"], parse_dates=[DATE_LABEL])


# make sure all of the dataframes cover the same period
svs_output = svs_output[
    (svs_output.date >= START_DATE) & (svs_output.date <= END_DATE)
].reset_index(drop=True)

obs_percolation = obs_percolation[
    (obs_percolation.date >= START_DATE) & (obs_percolation.date <= END_DATE)
].reset_index(drop=True)

obs_soil = obs_soil[
    (obs_soil.date >= START_DATE) & (obs_soil.date <= END_DATE)
].reset_index(drop=True)

obs_meteo = obs_meteo[
    (obs_meteo.date >= START_DATE) & (obs_meteo.date <= END_DATE)
].reset_index(drop=True)

assert svs_output[DATE_LABEL].nunique() == obs_percolation[DATE_LABEL].nunique() \
    == obs_soil[DATE_LABEL].nunique() == obs_meteo[DATE_LABEL].nunique()


# create a single column for soil moisture (%) and soil temperature (°C)
soil_moist_label = "soil_moisture"
svs_output[soil_moist_label] = svs_output[
    [F"WSOIL_{str(layer_nr)}" for layer_nr in svs_layers]
].mean(axis=1) * 100

soil_temp_label = "soil_temperature"
svs_output[soil_temp_label] = svs_output[
    [F"TPSOIL_{str(layer_nr)}" for layer_nr in svs_layers]
].mean(axis=1) - 273.15

# change unit of SNODP from m to cm
svs_output["SNODP"] *= 100

# for consistency, `enclosure` label will be renamed to `Experimental Plot`
new_enc_label = "Experimental Plot"
svs_output = svs_output.rename(columns={"enclosure": new_enc_label})

# select the columns of interest from the SVS output
# "DRAI" is the percolation (mm/day)
# "SNODP" is the snow depth over bare ground and low vegetation (cm)
interest_cols = [
    DATE_LABEL, new_enc_label, "member",
    soil_moist_label, soil_temp_label, "DRAI", "SNODP"
] 
svs_output = svs_output[interest_cols]

# create the ensemble average of the SVS output
# for each date-enclosure average over the ensemble members
svs_ensemble_avg = svs_output.groupby(
    [DATE_LABEL, new_enc_label], as_index=False
).mean(numeric_only=True)


# check if the number of ensemble members is correct
assert svs_output.groupby(
    [DATE_LABEL, new_enc_label], as_index=False
).count()["member"].unique() == [N_ENS], F"Number of ensemble members is not {N_ENS}!"


# ______________________________________________________________________ <<< >>>


# Deterministic Metrics

In [ ]:
# calc r2 and mbe for soil moisture 
determ_moisture = pd.merge(
    svs_ensemble_avg.loc[:, [DATE_LABEL, new_enc_label,  soil_moist_label]],
    obs_soil.loc[obs_soil["Variable"] == "Soil Volumetric Water Content", :],
    on=[DATE_LABEL, new_enc_label], how="inner"

# for each plot and sensor
).groupby([new_enc_label, "Sensor"]).apply(
    lambda x: pd.Series({
        "r2": x.loc[:, [soil_moist_label, "Value"]].corr().iloc[0, 1] ** 2,
        "mbe_percent": (x[soil_moist_label] - x["Value"]).mean()
    })
).reset_index()


# calc r2 and mbe for soil temperature
determ_temp = pd.merge(
    svs_ensemble_avg.loc[:, [DATE_LABEL, new_enc_label,  soil_temp_label]],
    obs_soil.loc[obs_soil["Variable"] == "Soil Temperature", :],
    on=[DATE_LABEL, new_enc_label], how="inner"

# for each plot and sensor
).groupby([new_enc_label, "Sensor"]).apply(
    lambda x: pd.Series({
        "r2": x.loc[:, [soil_temp_label, "Value"]].corr().iloc[0, 1] ** 2,
        "mbe_degC": (x[soil_temp_label] - x["Value"]).mean()
    })
).reset_index()


# calc r2 and mbe for percolation
determ_perc = pd.merge(
    svs_ensemble_avg.loc[:, [DATE_LABEL, new_enc_label,  "DRAI"]],
    obs_percolation,
    on=[DATE_LABEL, new_enc_label], how="inner"

# for each plot and Lysimeter
).groupby([new_enc_label, "Lysimeter"]).apply(
    lambda x: pd.Series({
        "r2": x.loc[:, ["DRAI", "Percolation (mm/day)"]].corr().iloc[0, 1] ** 2,
        "mbe_mm": (x["DRAI"] - x["Percolation (mm/day)"]).mean()
    })
).reset_index()


# calc r2 and mbe for snow depth
# only for one plot

# Calculate for first winter period
determ_snow_first = pd.merge(
    svs_ensemble_avg.loc[(svs_ensemble_avg["Experimental Plot"] == "E1") & 
                         (svs_ensemble_avg[DATE_LABEL].isin(first_winter)), 
                         [DATE_LABEL, "SNODP"]],
    obs_meteo[[DATE_LABEL, "Snow depth (cm)"]],
    on=DATE_LABEL, how="inner"
)

snow_r2_first = determ_snow_first.loc[:, ["SNODP", "Snow depth (cm)"]].corr().iloc[0, 1] ** 2
snow_mbe_first = (determ_snow_first["SNODP"] - determ_snow_first["Snow depth (cm)"]).mean()

# Calculate for second winter period
determ_snow_second = pd.merge(
    svs_ensemble_avg.loc[(svs_ensemble_avg["Experimental Plot"] == "E1") & 
                         (svs_ensemble_avg[DATE_LABEL].isin(second_winter)), 
                         [DATE_LABEL, "SNODP"]],
    obs_meteo[[DATE_LABEL, "Snow depth (cm)"]],
    on=DATE_LABEL, how="inner"
)

snow_r2_second = determ_snow_second.loc[:, ["SNODP", "Snow depth (cm)"]].corr().iloc[0, 1] ** 2
snow_mbe_second = (determ_snow_second["SNODP"] - determ_snow_second["Snow depth (cm)"]).mean()

# Average the metrics
average_snow_r2 = (snow_r2_first + snow_r2_second) / 2
average_snow_mbe = (snow_mbe_first + snow_mbe_second) / 2




# ------------------------------------------------------------------------------
# combine all the evaluation metrics into a single DataFrame -------------------

# Initiate an empty DataFrame to store the results
determ_eval_metrics = pd.DataFrame(columns=['Experimental Plot', 'Variable', 'Sensor/Lysimeter', 'r2', 'MBE'])

# Insert evaluation metrics for Soil Moisture
for index, row in determ_moisture.iterrows():
    new_row = pd.DataFrame({
        'Experimental Plot': [row[new_enc_label]],
        'Variable': ['Soil Moisture'],
        'Sensor/Lysimeter': [row["Sensor"]],
        'r2': [row['r2']],
        'MBE': [row['mbe_percent']]
    })
    determ_eval_metrics = pd.concat([determ_eval_metrics, new_row])

# Insert evaluation metrics for Soil Temperature
for index, row in determ_temp.iterrows():
    new_row = pd.DataFrame({
        'Experimental Plot': [row[new_enc_label]],
        'Variable': ['Soil Temperature'],
        'Sensor/Lysimeter': [row["Sensor"]],
        'r2': [row['r2']],
        'MBE': [row['mbe_degC']]
    })
    determ_eval_metrics = pd.concat([determ_eval_metrics, new_row])

# Insert evaluation metrics for Percolation
for index, row in determ_perc.iterrows():
    new_row = pd.DataFrame({
        'Experimental Plot': [row[new_enc_label]],
        'Variable': ['Percolation'],
        'Sensor/Lysimeter': [row["Lysimeter"]],
        'r2': [row['r2']],
        'MBE': [row['mbe_mm']]
    })
    determ_eval_metrics = pd.concat([determ_eval_metrics, new_row])

# Insert evaluation metrics for Snow Depth
new_row = pd.DataFrame({
    'Experimental Plot': [""],
    'Variable': ['Snow Depth'],
    'Sensor/Lysimeter': [None],
    'r2': [average_snow_r2],
    'MBE': [average_snow_mbe]
})
determ_eval_metrics = pd.concat([determ_eval_metrics, new_row])

# Reset the index
determ_eval_metrics = determ_eval_metrics.reset_index(drop=True)


# remove variables that are not needed anymore
del determ_moisture, determ_temp, determ_perc, determ_snow_first, determ_snow_second

# Probabilistic Metrics

In [ ]:
# ignore RuntimeWarnings
warnings.filterwarnings("ignore", category=RuntimeWarning, message="invalid value encountered in divide")

# calc CRPS for soil moisture
crps_moisture =( 

    # merge soil moisture ensemble output with observations
    pd.merge(
    svs_output.loc[:, [DATE_LABEL, new_enc_label,  soil_moist_label, 'member']],
    obs_soil.loc[obs_soil["Variable"] == "Soil Volumetric Water Content", [DATE_LABEL, new_enc_label, "Sensor", "Value"]],
    on=[DATE_LABEL, new_enc_label], how="inner"
)   
    # spread the ensemble members into columns
    .pivot_table(index=[DATE_LABEL, new_enc_label, "Sensor", "Value"], columns="member", values=soil_moist_label)
    .sort_index(level=0)
    .reset_index()

    # for each plot and sensor
    .groupby(["Experimental Plot", "Sensor"])

    # calculate the CRPS and benchmark CRPS
    .apply(lambda x: pd.concat([
        calculate_crps(x, N_ENS), benchmark_crps(x, N_ENS)
    ], axis=1))

    .reset_index()
    .drop(columns="level_2")

    # calculate the CRPS skill
    .assign(CRPSS=lambda x: 1 - x["CRPS"]/x["Benchmark CRPS"])
)

# calc CRPS for soil temperature
crps_temp =(
    # merge soil temperature ensemble output with observations
    pd.merge(
    svs_output.loc[:, [DATE_LABEL, new_enc_label,  soil_temp_label, 'member']],
    obs_soil.loc[obs_soil["Variable"] == "Soil Temperature", [DATE_LABEL, new_enc_label, "Sensor", "Value"]],
    on=[DATE_LABEL, new_enc_label], how="inner"
)
    # spread the ensemble members into columns
    .pivot_table(index=[DATE_LABEL, new_enc_label, "Sensor", "Value"], columns="member", values=soil_temp_label)
    .sort_index(level=0)
    .reset_index()

    # for each plot and sensor
    .groupby(["Experimental Plot", "Sensor"])

    # calculate the CRPS and benchmark CRPS
    .apply(lambda x: pd.concat([
        calculate_crps(x, N_ENS), benchmark_crps(x, N_ENS)
    ], axis=1))

    .reset_index()
    .drop(columns="level_2")

    # calculate the CRPS skill
    .assign(CRPSS=lambda x: 1 - x["CRPS"]/x["Benchmark CRPS"])
)

# calc CRPS for percolation
crps_perc = (
    # merge percolation ensemble output with observations
    pd.merge(
    svs_output.loc[:, [DATE_LABEL, new_enc_label,  "DRAI", 'member']],
    obs_percolation.loc[:, [DATE_LABEL, new_enc_label, "Lysimeter", "Percolation (mm/day)"]],
    on=[DATE_LABEL, new_enc_label], how="inner"
)
    # spread the ensemble members into columns
    .pivot_table(index=[DATE_LABEL, new_enc_label, "Lysimeter", "Percolation (mm/day)"], columns="member", values="DRAI")
    .sort_index(level=0)
    .reset_index()

    # for each plot and lysimeter
    .groupby(["Experimental Plot", "Lysimeter"])

    # calculate the CRPS and benchmark CRPS
    .apply(lambda x: pd.concat([
        calculate_crps(x, N_ENS, obs_column="Percolation (mm/day)"),
        benchmark_crps(x, N_ENS, obs_column="Percolation (mm/day)")
    ], axis=1))

    .reset_index()
    .drop(columns="level_2")

    # calculate the CRPS skill
    .assign(CRPSS=lambda x: 1 - x["CRPS"]/x["Benchmark CRPS"])
)

# calc CRPS for snow depth

# first winter
crps_snow_first = (
    # merge snow depth ensemble output with observations
    pd.merge(
    svs_output.loc[
        (svs_output[DATE_LABEL].isin(first_winter)) & 
        (svs_output[new_enc_label] == "E1"), 
        [DATE_LABEL, new_enc_label,  "SNODP", 'member']
    ],
    obs_meteo.loc[:, [DATE_LABEL, "Snow depth (cm)"]],
    on=[DATE_LABEL], how="inner"
)
    # spread the ensemble members into columns
    .pivot_table(index=[DATE_LABEL, new_enc_label, "Snow depth (cm)"], columns="member", values="SNODP")
    .sort_index(level=0)
    .reset_index()

    # calculate the CRPS and benchmark CRPS
    .pipe(lambda x: pd.concat([
        calculate_crps(x, N_ENS, obs_column="Snow depth (cm)"),
        benchmark_crps(x, N_ENS, obs_column="Snow depth (cm)")
    ], axis=1))

    # calculate the CRPS skill
    .assign(CRPSS=lambda x: 1 - x["CRPS"]/x["Benchmark CRPS"])
)

# second winter
crps_snow_second = (
    # merge snow depth ensemble output with observations
    pd.merge(
    svs_output.loc[
        (svs_output[DATE_LABEL].isin(second_winter)) &
        (svs_output[new_enc_label] == "E1"),
        [DATE_LABEL, new_enc_label,  "SNODP", 'member']
    ],
    obs_meteo.loc[:, [DATE_LABEL, "Snow depth (cm)"]],
    on=[DATE_LABEL], how="inner"
)
    # spread the ensemble members into columns
    .pivot_table(index=[DATE_LABEL, new_enc_label, "Snow depth (cm)"], columns="member", values="SNODP")
    .sort_index(level=0)
    .reset_index()

    # calculate the CRPS and benchmark CRPS
    .pipe(lambda x: pd.concat([
        calculate_crps(x, N_ENS, obs_column="Snow depth (cm)"),
        benchmark_crps(x, N_ENS, obs_column="Snow depth (cm)")
    ], axis=1))

    # calculate the CRPS skill
    .assign(CRPSS=lambda x: 1 - x["CRPS"]/x["Benchmark CRPS"])
)

average_snow_crps = (crps_snow_first["CRPS"].values + crps_snow_second["CRPS"].values) / 2
average_snow_benchmark = (crps_snow_first["Benchmark CRPS"].values + crps_snow_second["Benchmark CRPS"].values) / 2
average_snow_crpss = 1 - average_snow_crps / average_snow_benchmark



# ------------------------------------------------------------------------------
# combine all CRPS dataframes --------------------------------------------------
# Initiate an empty DataFrame to store the results
crps_eval_metrics = pd.DataFrame(columns=['Experimental Plot', 'Variable', 'Sensor/Lysimeter', 'CRPS', 'CRPSS'])

# Insert evaluation metrics for Soil Moisture
for index, row in crps_moisture.iterrows():
    new_row = pd.DataFrame({
        'Experimental Plot': [row['Experimental Plot']],
        'Variable': ['Soil Moisture'],
        'Sensor/Lysimeter': [row["Sensor"]],
        'CRPS': [row['CRPS']],
        'CRPSS': [row['CRPSS']]
    })
    crps_eval_metrics = pd.concat([crps_eval_metrics, new_row])

# Insert evaluation metrics for Soil Temperature
for index, row in crps_temp.iterrows():
    new_row = pd.DataFrame({
        'Experimental Plot': [row['Experimental Plot']],
        'Variable': ['Soil Temperature'],
        'Sensor/Lysimeter': [row["Sensor"]],
        'CRPS': [row['CRPS']],
        'CRPSS': [row['CRPSS']]
    })
    crps_eval_metrics = pd.concat([crps_eval_metrics, new_row])

# Insert evaluation metrics for Percolation
for index, row in crps_perc.iterrows():
    new_row = pd.DataFrame({
        'Experimental Plot': [row['Experimental Plot']],
        'Variable': ['Percolation'],
        'Sensor/Lysimeter': [row["Lysimeter"]],
        'CRPS': [row['CRPS']],
        'CRPSS': [row['CRPSS']]
    })
    crps_eval_metrics = pd.concat([crps_eval_metrics, new_row])

# Insert average evaluation metrics for Snow Depth
new_row = pd.DataFrame({
    'Experimental Plot': [""],
    'Variable': ['Snow Depth'],
    'Sensor/Lysimeter': [None],
    'CRPS': average_snow_crps,
    'CRPSS': average_snow_crpss
})
crps_eval_metrics = pd.concat([crps_eval_metrics, new_row])

# Reset the index
crps_eval_metrics = crps_eval_metrics.reset_index(drop=True)

# remove variables that are not needed anymore
del crps_moisture, crps_temp, crps_perc

# Combine both dataframes into one for easy plotting
df_combined = pd.merge(determ_eval_metrics, crps_eval_metrics, on=['Experimental Plot', 'Variable', 'Sensor/Lysimeter'], how='outer')

# average over `Sensor/Lysimeter` for each variable
df_combined_ave = df_combined.groupby(['Experimental Plot', 'Variable']).mean(numeric_only=True).reset_index()

# reset to default warning settings
warnings.resetwarnings()

# Plot the results

In [ ]:
# plotting parameters
figsize = (11, 11)
fontsize = 70

dfplot = df_combined_ave.melt(
    id_vars=["Experimental Plot", "Variable"],
    value_vars=["CRPS", "CRPSS", "r2", "MBE"],
    var_name="Metric", value_name="Value"
).copy()

# create 4 colors for the 4 experimental plots
colors = sns.color_palette("colorblind", n_colors=4)

# create a column with the colors
dfplot["Color"] = dfplot["Experimental Plot"].map({
    "E1": colors[0],
    "E2": colors[1],
    "E3": colors[2],
    "": colors[3]
})

for vbl in dfplot["Variable"].unique():

    for met in dfplot["Metric"].unique():

        dfplot2 = dfplot.loc[
            (dfplot["Variable"] == vbl) &
            (dfplot["Metric"] == met)
        ].copy()


        fig, ax = plt.subplots(1, 1, figsize=figsize)

        # set the figure style
        sns.set_style("white")
        sns.despine()
        ax.grid(False)

        # plot the data
        xorder = ["E1", "E2", "E3"]
        if vbl == "Snow Depth":
            xorder = ["", "E1", "E2", "E3"]

        sns.barplot(
            data=dfplot2,
            x="Experimental Plot", y="Value",
            hue="Experimental Plot", palette=dfplot2["Color"].unique(), ax=ax,
            errorbar=None, order=xorder, dodge=False,
        )

       

        # add the yvalue as text above each bar
    
        for p in ax.patches:
            ax.annotate(
                F"{p.get_height():.2f}",
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', fontsize=fontsize//1.15, color='black', xytext=(0, 5),
                textcoords='offset points'
            )

        # y-axis
        yx_lab = F"{met}"

        # if met is r2, use R superscript 2
        if met == "r2":
            yx_lab = r"R$^2$"

        if (yx_unit := get_units(vbl, met)) != "-":

            # if unit is mm, then make it mm divided by day like the latex \
            if yx_unit == "mm":
                yx_unit = r'$\frac{mm}{day}$'
            
            yx_lab = F"{yx_lab} ({yx_unit})"
        
        # add gap between yaxis label and yaxis
        ax.set_ylabel(yx_lab, fontsize=fontsize, fontweight='bold', labelpad=1)

        # set font size for xticks and yticks; also make bold and black
        ax.tick_params(axis='both', which='major', labelsize=fontsize)

        # no xlabel
        ax.set_xlabel(None)

        # no yticks
        ax.set_yticklabels([])


        
        # if its snow, no xticks
        if vbl == "Snow Depth":
            ax.set_xticklabels([])

        # no legend
        ax.legend([],[], frameon=False)

        # set ylim
        ylim = get_axis_limits(vbl, met)
        ax.set_ylim(ylim)

        # if met is MBE, then add a thin horizontal line at 0
        if met == "MBE":
            ax.axhline(0, color='black', linewidth=0.5, linestyle='--')

        # save path 
        spath = Path(F"./output/figures/{vbl}_{met}.jpg")
        # plt.savefig(spath, dpi=600, bbox_inches='tight', pad_inches=0.3)
        plt.show()

        # raise Exception